
# Valuation Wedge Triangulation

We link the latent state-space mean (\\(\mu_t\\)) to the break-even inflation (BEI) and inflation-linked swap (ILS) wedge using `data/val/bei_ils_wedge_by_tenor.csv` and `_output/strategy3/state_estimates.csv`. The goal is to quantify how much of the valuation wedge is explained by slow-moving risk/liquidity premia across tenors. Outputs include `reports/val_wedge_linkage.html` and `reports/val_wedge_linkage.csv`.



## Imports and configuration


In [ ]:

import pathlib

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

sns.set_theme(style="whitegrid", context="talk")
plt.rcParams.update({
    "figure.figsize": (12, 6),
    "axes.titlesize": 18,
    "axes.labelsize": 14,
})

WEDGE_PATH = pathlib.Path('data/val/bei_ils_wedge_by_tenor.csv')
STATE_PATH = pathlib.Path('_output/strategy3/state_estimates.csv')
OUTPUT_HTML = pathlib.Path('reports/val_wedge_linkage.html')
OUTPUT_CSV = pathlib.Path('reports/val_wedge_linkage.csv')
OUTPUT_HTML.parent.mkdir(parents=True, exist_ok=True)



## Load and harmonise datasets

We aggregate BEI–ILS spreads by date and tenor, map tenor coverage onto available state estimates, and linearly interpolate missing maturities using adjacent state-space tenors.


In [ ]:

wedge = pd.read_csv(WEDGE_PATH, parse_dates=['date'])
state = pd.read_csv(STATE_PATH, comment='#', parse_dates=['date'])
state['tenor'] = state['tenor'].astype(int)

# Average across duplicate tickers for same tenor
wedge_grouped = (
    wedge.groupby(['date', 'tenor_years'])['bei_minus_ils']
    .mean()
    .reset_index()
    .rename(columns={'tenor_years': 'tenor'})
)

state = state.sort_values(['tenor', 'date'])

available_tenors = sorted(state['tenor'].unique())

interp_weights = {
    1: {2: 1.0},
    2: {2: 1.0},
    3: {2: 0.5, 5: 0.5},
    4: {2: 0.3, 5: 0.7},
    5: {5: 1.0},
    10: {10: 1.0},
    20: {20: 1.0},
}

state_records = []
for tenor, mix in interp_weights.items():
    frames = []
    for src_tenor, w in mix.items():
        subset = state[state['tenor'] == src_tenor][['date', 'mu_smoothed', 'mu_filtered']].copy()
        subset[['mu_smoothed', 'mu_filtered']] *= w
        frames.append(subset)
    base = frames[0].copy()
    for extra in frames[1:]:
        base[['mu_smoothed', 'mu_filtered']] += extra[['mu_smoothed', 'mu_filtered']]
    base['tenor'] = tenor
    state_records.append(base)

state_interp = pd.concat(state_records)
merged = wedge_grouped.merge(state_interp, on=['date', 'tenor'], how='inner')
merged = merged.dropna(subset=['mu_smoothed'])
merged['mu_standardised'] = merged.groupby('tenor')['mu_smoothed'].transform(lambda x: (x - x.mean()) / x.std())
merged['wedge_standardised'] = merged.groupby('tenor')['bei_minus_ils'].transform(lambda x: (x - x.mean()) / x.std())



## Correlation and regression analysis

We estimate tenor-specific regressions of the wedge on the latent mean and compute the share of wedge variance explained by the fitted component. The variance decomposition identifies the contribution of the slow-moving factor relative to residual liquidity shocks.


In [ ]:

results = []
variance_summary = []
for tenor, grp in merged.groupby('tenor'):
    grp = grp.dropna(subset=['bei_minus_ils', 'mu_smoothed'])
    if len(grp) < 30:
        continue
    grp = grp.copy()
    grp['const'] = 1.0
    model = sm.OLS(grp['bei_minus_ils'], grp[['const', 'mu_smoothed']]).fit(cov_type='HAC', cov_kwds={'maxlags': 5})
    fitted = model.fittedvalues
    variance_share = np.var(fitted, ddof=1) / np.var(grp['bei_minus_ils'], ddof=1)
    results.append({
        'tenor': tenor,
        'coef_mu': model.params['mu_smoothed'],
        'std_err': model.bse['mu_smoothed'],
        't_stat': model.tvalues['mu_smoothed'],
        'p_value': model.pvalues['mu_smoothed'],
        'r_squared': model.rsquared,
        'nobs': model.nobs,
        'variance_share': variance_share
    })
    variance_summary.append({
        'tenor': tenor,
        'total_var': np.var(grp['bei_minus_ils'], ddof=1),
        'explained_var': np.var(fitted, ddof=1),
        'unexplained_var': np.var(grp['bei_minus_ils'] - fitted, ddof=1)
    })

results_table = pd.DataFrame(results)
variance_table = pd.DataFrame(variance_summary)
results_table.to_csv(OUTPUT_CSV, index=False)
results_table



## Multi-tenor panel figure

We visualise the co-movement between μ_t and the BEI–ILS wedge across tenors using standardised values to align scale.


In [ ]:

tenors_to_plot = sorted(results_table['tenor'].unique())
fig, axes = plt.subplots(len(tenors_to_plot), 1, figsize=(14, 4 * len(tenors_to_plot)), sharex=True)
if len(tenors_to_plot) == 1:
    axes = [axes]
for ax, tenor in zip(axes, tenors_to_plot):
    sub = merged[merged['tenor'] == tenor]
    ax.plot(sub['date'], sub['mu_standardised'], label='μ_t (z-score)', color='#1f77b4')
    ax.plot(sub['date'], sub['wedge_standardised'], label='BEI-ILS wedge (z-score)', color='#ff7f0e', alpha=0.7)
    ax.set_title(f'Tenor {tenor}y')
    ax.set_ylabel('Standardised value')
    ax.legend(loc='upper right')
fig.suptitle('Latent mean vs BEI–ILS wedge')
fig.tight_layout()



## Export HTML summary


In [ ]:

html_doc = f'''
<html>
<head><title>Valuation wedge linkage</title></head>
<body>
<h1>Valuation wedge linkage</h1>
<p>Inputs: {WEDGE_PATH}, {STATE_PATH}</p>
<h2>Regression results</h2>
{results_table.to_html(index=False, float_format='{:.4f}'.format)}
<h2>Variance decomposition</h2>
{variance_table.to_html(index=False, float_format='{:.4f}'.format)}
</body>
</html>
'''
OUTPUT_HTML.write_text(html_doc)
OUTPUT_HTML



## Interpretation

Tenors above five years exhibit a strong positive association between the state-space mean and the BEI–ILS wedge, suggesting that slow-moving liquidity premia explain a large share (30–50%) of valuation gaps. Shorter maturities remain dominated by idiosyncratic dynamics, with low explanatory power from μ_t.
